<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/6_Federated_MLP256_newProcessing_HighRecall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overview: https://www.tensorflow.org/federated

Image classification tutorial: https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification

# Installation

In [1]:
!pip install --quiet fastai==2.2.5
!pip install --quiet folium==0.2.1
!pip install --quiet imgaug==0.2.5
!pip install --quiet tensorflow==2.3.0
!pip install --quiet tensorflow_federated==0.17.0
!pip install --quiet --upgrade nest_asyncio

     |████████████████████████████████| 194kB 19.6MB/s 
     |████████████████████████████████| 61kB 10.8MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 563kB 19.4MB/s 
     |████████████████████████████████| 320.4MB 49kB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 460kB 53.9MB/s 
     |████████████████████████████████| 522kB 18.7MB/s 
     |████████████████████████████████| 174kB 19.1MB/s 
     |████████████████████████████████| 153kB 41.3MB/s 
     |████████████████████████████████| 3.0MB 56.7MB/s 
     |████████████████████████████████| 112kB 52.9MB/s 
     |████████████████████████████████| 1.1MB 39.5MB/s 


In [2]:
import nest_asyncio
nest_asyncio.apply()
%load_ext tensorboard

In [3]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd 
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import data

I dati vengono importati e poi splittati in train e test. 

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')

In [6]:
data = mf.new_processing(data)
data = pd.get_dummies(data)

In [7]:
col_name = mf.get_col(data)

In [8]:
print('Rate safe/fraud:', (1/mf.ratio(data)).round(3))

Rate safe/fraud: 0.036


In [9]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2)

Smote: https://imbalanced-learn.org/stable/generated/imblearn.over_sampling.SMOTE.html

RandomUnderSampler: https://imbalanced-learn.org/stable/generated/imblearn.under_sampling.RandomUnderSampler.html

In [10]:
def underSampling(data, frac_under=0.1):
  from imblearn.under_sampling import RandomUnderSampler 
  us = RandomUnderSampler(sampling_strategy=frac_under, random_state=42)
  y = data['isFraud']
  X = data.drop(columns = ['isFraud'])
  X_us, y_us = us.fit_resample(X, y)
  return X_us, y_us

def overSampling(X, y, frac_over=0.3):
  from imblearn.over_sampling import SMOTE
  sm = SMOTE(sampling_strategy=frac_over, random_state=42)
  X_sm, y_sm = sm.fit_resample(X, y)  
  return X_sm, y_sm

def mergeResult(X, y, col_name):
  y_res = np.ndarray(shape=(np.shape(y)[0],1), buffer = y)
  data = np.concatenate((X,y_res), axis = 1)
  col_name.append('isFraud')
  dataset = pd.DataFrame(data=data, columns=col_name)  
  return dataset

def rate(y):
  n_fraud = np.count_nonzero(y == 1)
  n_safe = np.shape(y)[0] - n_fraud
  return f'Rate safe/fraud: {n_safe/n_fraud}'

In [11]:
X, y = underSampling(train_data)
rate(y)

'Rate safe/fraud: 10.0'

In [12]:
X, y = overSampling(X, y)
rate(y)

'Rate safe/fraud: 3.3333333333333335'

In [13]:
train_data = mergeResult(X, y, col_name)

In [14]:
# print('Rate safe/fraud:', (mf.ratio(train_data)).round(3))
print('Dataset size:', train_data.shape[0])

Dataset size: 214305


In [15]:
del data, X, y

In [16]:
train_data.shape

(214305, 734)

# Convert data

Il dataset deve essere convertito in un tensore, con componenti (feature_vector, label). 

In [17]:
def to_tensor(data, n_clients = 4):
  shuffled = data.sample(frac=1)
  result = np.array_split(shuffled, n_clients)  

  res = []
  label = []

  for dataset in result:
    label.append(dataset['isFraud'])
    res.append(dataset.drop(columns = ['isFraud']))

  dataset = {}
  for i in range(n_clients):
    dataset[i] = tf.data.Dataset.from_tensor_slices((res[i], label[i]))
  return dataset

In [18]:
dataset = to_tensor(train_data)
test_set = to_tensor(test_data)

In [19]:
del train_data, test_data

# Federated data

Si definisce la funzione di preprocessing del dataset, che serve a creare l'OrderedDict, su cui si andranno a creare le batch necessarie per il training del modello. 

In [20]:
NUM_CLIENTS = 4
NUM_EPOCHS = 10
BATCH_SIZE = 250
SHUFFLE_BUFFER = 10
PREFETCH_BUFFER = 10

def preprocess(dataset):
  def batch_format_fn(e1, e2):
    return collections.OrderedDict(
        x = tf.cast(e1, tf.float32),
        y = tf.cast(e2, tf.int32))
  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

preprocessed_example_dataset = preprocess(dataset[0])

I dati federati sono una lista di dataset divisi per cliente. 

In [21]:
def make_federated_data(dataset):
  federated = []
  for i in dataset:
    federated.append(preprocess(dataset[i]))
  return federated

federated_train_data = make_federated_data(dataset)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

Number of client datasets: 4
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 733)), (y, (None,))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>


In [22]:
del dataset

# Model creation and training

Creazione della rete neurale che sarà trainata. 

Viene anche definita la model function, in cui si specifica il modello, il tipo di input, la loss e le metriche da utilizzare. 

Infine si costruisce il processo di averaging, specificando l'optimizer da usare, cioè SGD, e il learning rate del server e del client. 

BatchNormalization Layer: https://stackoverflow.com/questions/56514398/why-the-tensorflow-federated-performance-is-worse-than-single-keras-model

In [30]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input, BatchNormalization

def create_keras_model():
  model = Sequential()
  model.add(Input(shape=(733,)))   
  model.add(Dense(256, activation='relu')) 
  # model.add(BatchNormalization())
  model.add(Dropout(0.1))
  model.add(Dense(1, activation='sigmoid'))
  return model

def model_fn():
  soglia = 0.5
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tf.keras.metrics.BinaryAccuracy(), 
               tf.keras.metrics.Recall(thresholds=soglia),
               tf.keras.metrics.Precision(thresholds=soglia)])
  
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,  
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.4), #0.05
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1)) 

Training del modello. 

Gpu usage: https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=Y04m-jvKRDsJ

In [31]:
NUM_ROUNDS = 40
state = iterative_process.initialize()
for round_num in range(1, NUM_ROUNDS + 1):
  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_accuracy', 0.8215815), ('recall', 0.39932463), ('precision', 0.6984153), ('loss', 0.40831912)]))])
round  2, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_accuracy', 0.83977836), ('recall', 0.48099485), ('precision', 0.7328281), ('loss', 0.3741637)]))])
round  3, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_accuracy', 0.8500811), ('recall', 0.521377), ('precision', 0.75317883), ('loss', 0.3543922)]))])
round  4, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('binary_accuracy', 0.85749656), ('rec

# Model evaluation

Evaluation del modello sui test data. 

In [32]:
evaluation = tff.learning.build_federated_evaluation(model_fn)
train_metrics = evaluation(state.model, federated_train_data)

In [33]:
federated_test_data = make_federated_data(test_set)

In [34]:
test_metrics = evaluation(state.model, federated_test_data)
str(test_metrics)

"OrderedDict([('binary_accuracy', 0.8788895), ('recall', 0.73497725), ('precision', 0.18877205), ('loss', 0.30097085)])"

Board di tensorboad, per visualizzare la loss e le metriche in modo interattivo. 

In [35]:
# logdir = "/tmp/logs/scalars/training/"
# summary_writer = tf.summary.create_file_writer(logdir)
# state = iterative_process.initialize()
# with summary_writer.as_default():
#   for round_num in range(1, NUM_ROUNDS):
#     state, metrics = iterative_process.next(state, federated_train_data)
#     for name, value in metrics['train'].items():
#       tf.summary.scalar(name, value, step=round_num)

In [36]:
# !ls {logdir}
# %tensorboard --logdir {logdir} --port=0